## Work on Ocean Data CM2 0.25 - 3D Data

In [1]:
import xarray as xr
import numpy as np
import cmor

In [2]:
ds = xr.open_dataset("Test_data/cj877/history/ocn/ocean-3d-pot_temp-1-monthly-mean-ym_0312_07.nc", decode_times=False)

In [4]:
varname = "pot_temp"
var_data = ds[varname]

In [5]:
i_axis = ds[varname]["xt_ocean"]
j_axis = ds[varname]["yt_ocean"]

In [6]:
x = np.arange(i_axis.size, dtype="float")
x_bnds = np.array([[x_ - 0.5, x_ + 0.5] for x_ in x])
y = np.arange(j_axis.size, dtype="float")
y_bnds = np.array([[y_ - 0.5, y_ + 0.5] for y_ in y])

### MOM6 Supergrid

In [7]:
grid_filepath = "./grids/access-om2/input_20201102/mom_025deg/ocean_hgrid.nc"

In [8]:
grid = xr.open_dataset(grid_filepath)

In [9]:
# T point locations
xt = grid['x'][1::2,1::2]
yt = grid['y'][1::2,1::2]
# Corner point locations
xq = grid['x'][::2,::2]
yq = grid['y'][::2,::2]
# U point locations
xu = grid['x'][1::2,::2]
yu = grid['y'][1::2,::2]
# V point locations
xv = grid['x'][::2,1::2]
yv = grid['y'][::2,1::2]

In [10]:
lat = yt.values
lon = (xt.values + 360) % 360

In [11]:
lat_bnds = np.zeros((*yt.shape, 4))
lon_bnds = np.zeros((*xt.shape, 4))

In [12]:
lat_bnds[..., 0] = yq[1:, :-1]  # SW corner
lat_bnds[..., 1] = yq[1:, 1:]   # SE corner
lat_bnds[..., 2] = yq[:-1, 1:]  # NE corner
lat_bnds[..., 3] = yq[:-1, :-1] # NW corner

xq = (xq + 360) % 360
lon_bnds[..., 0] = xq[1:, :-1]  # SW corner
lon_bnds[..., 1] = xq[1:, 1:]   # SE corner
lon_bnds[..., 2] = xq[:-1, 1:]  # NE corner
lon_bnds[..., 3] = xq[:-1, :-1] # NW corner

In [13]:
time = ds["time"].values
time_bnds = ds["time_bnds"].values

In [14]:
ipth = opth = 'Test'
cmor.setup(inpath=ipth,
           set_verbosity=cmor.CMOR_NORMAL,
           netcdf_file_action=cmor.CMOR_REPLACE,
           exit_control=cmor.CMOR_EXIT_ON_MAJOR)
cmor.dataset_json('CMOR_input_example.json')

0

In [15]:
# First, load the grids table to set up x and y axes and the lat-long grid
grid_table_id = cmor.load_table("cmip6-cmor-tables/Tables/CMIP6_grids.json")
cmor.set_table(grid_table_id)

0

In [16]:
lat.shape, lat_bnds.shape, var_data.shape

((1080, 1440), (1080, 1440, 4), (6, 50, 1080, 1440))

In [17]:
yaxis_id = cmor.axis(table_entry="j_index", units="1", coord_vals=y, cell_bounds=y_bnds)

In [18]:
xaxis_id = cmor.axis(table_entry="i_index", units="1", coord_vals=x, cell_bounds=x_bnds)

In [19]:
grid_id = cmor.grid(axis_ids=np.array([yaxis_id, xaxis_id]), latitude=lat, longitude=lon, latitude_vertices=lat_bnds, longitude_vertices=lon_bnds)

In [20]:
# Now, load the Omon table to set up the time axis and variable
omon_table_id = cmor.load_table('./cmip6-cmor-tables/Tables/CMIP6_Omon.json')
cmor.set_table(omon_table_id)

0

In [21]:
time_units = ds["time"].units
time_axis_id = cmor.axis(table_entry='time',
                         units=time_units,
                         coord_vals=time,
                         cell_bounds=time_bnds)

In [22]:
depths = ds["st_ocean"].values
depths_bnds = ds["st_edges_ocean"].values
depth_axis_id = cmor.axis(table_entry="depth_coord",
                          units="m",
                          coord_vals=depths,
                          cell_bounds=depths_bnds)

In [23]:
var_id = cmor.variable(table_entry='thetao',
                       units='degC',
                       axis_ids=[grid_id, time_axis_id, depth_axis_id])

In [24]:
var_data = np.moveaxis(var_data.values, 0, -1)
var_data = np.moveaxis(var_data, 0, -1)

In [25]:
#var_data = var_data#[..., :2]

In [26]:
var_data.shape

(1080, 1440, 6, 50)

In [27]:
cmor.write(var_id, var_data, 2)


C Traceback:
In function: _CV_CompareNoParent
! called from: _CV_checkParentExpID
! called from: cmor_write
! 

!!!!!!!!!!!!!!!!!!!!!!!!!
!
! Warning: Your input attribute parent_mip_era with value "CMIP6" 
! will be replaced with value "no parent".
! 
!
!!!!!!!!!!!!!!!!!!!!!!!!!



0


C Traceback:
In function: _CV_CompareNoParent
! called from: _CV_checkParentExpID
! called from: cmor_write
! 

!!!!!!!!!!!!!!!!!!!!!!!!!
!
! Warning: Your input attribute parent_source_id with value "PCMDI-test-1-0" 
! will be replaced with value "no parent".
! 
!
!!!!!!!!!!!!!!!!!!!!!!!!!


C Traceback:
In function: _CV_CompareNoParent
! called from: _CV_checkParentExpID
! called from: cmor_write
! 

!!!!!!!!!!!!!!!!!!!!!!!!!
!
! Warning: Your input attribute parent_time_units with value "days since 1850-01-01" 
! will be replaced with value "no parent".
! 
!
!!!!!!!!!!!!!!!!!!!!!!!!!


C Traceback:
In function: _CV_CompareNoParent
! called from: _CV_checkParentExpID
! called from: cmor_write
! 

!!!!!!!!!!!!!!!!!!!!!!!!!
!
! Warning: Your input attribute parent_variant_label with value "r3i1p1f1" 
! will be replaced with value "no parent".
! 
!
!!!!!!!!!!!!!!!!!!!!!!!!!


C Traceback:
In function: _CV_CompareNoParent
! called from: _CV_checkParentExpID
! called from: cmor_write
! 


In [28]:
filename = cmor.close(var_id, file_name=True)
print("Stored in:", filename)
cmor.close()

Stored in: CMIP6/CMIP6/ISMIP6/PCMDI/PCMDI-test-1-0/piControl-withism/r3i1p1f1/Omon/thetao/gn/v20250219/thetao_Omon_PCMDI-test-1-0_piControl-withism_r3i1p1f1_gn_031701-031702.nc



C Traceback:
In function: cmor_close_variable
! 

!!!!!!!!!!!!!!!!!!!!!!!!!
!
! Warning: while closing variable 4 (thetao, table Omon)
! we noticed you wrote 2 time steps for the variable,
! but its time axis 0 (time) has 6 time steps
!
!!!!!!!!!!!!!!!!!!!!!!!!!

! ------
! CMOR is now closed.
! ------
! During execution we encountered:
!   7 Warning(s)
!   0 Error(s)
! ------
! Please review them.
! ------
! 


0